Learn  the  optimal  policy  for  the  frozen  lake environment  using  the  Policy  Iteration  vs  the Value Iteration technique.

In [1]:
import sys
!{sys.executable} -m pip install gym

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
import gym
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
from gym import envs
print(envs.registry.all())

dict_values([EnvSpec(id='CartPole-v0', entry_point='gym.envs.classic_control.cartpole:CartPoleEnv', reward_threshold=195.0, nondeterministic=False, max_episode_steps=200, order_enforce=True, autoreset=False, disable_env_checker=False, new_step_api=False, kwargs={}, namespace=None, name='CartPole', version=0), EnvSpec(id='CartPole-v1', entry_point='gym.envs.classic_control.cartpole:CartPoleEnv', reward_threshold=475.0, nondeterministic=False, max_episode_steps=500, order_enforce=True, autoreset=False, disable_env_checker=False, new_step_api=False, kwargs={}, namespace=None, name='CartPole', version=1), EnvSpec(id='MountainCar-v0', entry_point='gym.envs.classic_control.mountain_car:MountainCarEnv', reward_threshold=-110.0, nondeterministic=False, max_episode_steps=200, order_enforce=True, autoreset=False, disable_env_checker=False, new_step_api=False, kwargs={}, namespace=None, name='MountainCar', version=0), EnvSpec(id='MountainCarContinuous-v0', entry_point='gym.envs.classic_control.co

/usr/local/lib/python3.9/dist-packages/gym/envs/registration.py:421: UserWarning: WARN: The `registry.all` method is deprecated. Please use `registry.values` instead.
  logger.warn(


In [ ]:
def run_episodes(environment, n_episodes, policy, display=True):
    wins = 0
    total_reward = 0
    for episode in range(n_episodes):
        terminated = False
        state = environment.reset()
        while not terminated:
            
            if isinstance(policy, str) and policy == 'random':
                action = environment.action_space.sample()
            else:
                action = np.argmax(policy[state])

            
            next_state, reward, terminated, info = environment.step(action)

            
            if episode==1 and display:
                print("Action:")
                environment.render() 
            
            total_reward += reward
            
            state = next_state
            
            if terminated and reward == 1.0:
                wins += 1
    average_reward = total_reward / n_episodes
    return wins, total_reward, average_reward

In [ ]:
# Load a Frozen Lake environment
env = gym.make('FrozenLake-v1')
# Number of episodes to play
n_episodes = 5000
# First episode plotted as a sample episode
print('First episode:')
wins, total_reward, average_reward = run_episodes(env, n_episodes, policy="random", display=False)
print('------------------------------------')
print('Summary:')
print(f'- number of wins over {n_episodes} episodes = {wins}')
print(f'- average reward over {n_episodes} episodes = {average_reward}')

/usr/local/lib/python3.9/dist-packages/gym/core.py:317: DeprecationWarning: WARN: Initializing wrapper in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(
/usr/local/lib/python3.9/dist-packages/gym/wrappers/step_api_compatibility.py:39: DeprecationWarning: WARN: Initializing environment in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(


First episode:
------------------------------------
Summary:
- number of wins over 5000 episodes = 65
- average reward over 5000 episodes = 0.013


In [ ]:
!pip install pygame 

import os
os.environ['SDL_VIDEODRIVER']='dummy'
import pygame
pygame.display.set_mode((640,480))

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


<Surface(640x480x32 SW)>

In [ ]:
env.reset()
env.render()

/usr/local/lib/python3.9/dist-packages/gym/core.py:49: DeprecationWarning: WARN: You are calling render method, but you didn't specified the argument render_mode at environment initialization. To maintain backward compatibility, the environment will render in human mode.
If you want to render in human mode, initialize the environment in this way: gym.make('EnvName', render_mode='human') and don't call the render method.
See here for more information: https://www.gymlibrary.ml/content/api/
  deprecation(


1.Create a Policy Iteration function with the following parameters

> •policy: 2D array of a size n(S) x n(A), each cell represents a probability of taking action a in state s.

> •environment: Initialized OpenAI gym environment object•discount_factor: MDP discount factor


>•theta:  A  threshold  of  a  value  function  change.  Once  the  update  to  value function is below this number


> •max_iterations: Maximum number of iterations

In [ ]:
def policy_evaluation(policy, environment, discount_factor=1.0, theta=1e-9, max_iterations=1e9):
    n_states = environment.observation_space.n
    n_actions = environment.action_space.n
    # Number of evaluation iterations
    evaluation_iterations = 1
    # Initialize a value function for each state as zero
    V = np.zeros(n_states)
    # Repeat until change in value is below the threshold
    for i in range(int(max_iterations)):
        # Initialize a change of value function as zero
        delta = 0
        # Iterate though each state
        for state in range(n_states):
            v = V[state]
            v_a = []
            for action in range(n_actions):
                v_x = 0
                for (p, new_state, reward, terminating) in environment.P[state][action]:
                    v_x += policy[state][action] * p * (reward + V[new_state])
                v_a.append(v_x)

                # Update value function
            V[state] = max(v_a)
                
            # Calculate the absolute change of value function
            delta = max(delta, np.abs(V[state] - v))
            # Update value function
            #V[state] = v
        evaluation_iterations += 1

        # Terminate if value change is insignificant
        if delta < theta:
            # TODO - check how many iterations
            print(f'Policy-evaluation converged at iteration #{evaluation_iterations}')
            return V

In [ ]:
def one_step_lookahead(environment, state, V, discount_factor):
    n_states = environment.observation_space.n
    n_actions = environment.action_space.n
    action_values = np.zeros(n_actions)
    for action in range(n_actions):
        for probability, next_state, reward, terminated in environment.P[state][action]:
            action_values[action] += probability * (reward + discount_factor * V[next_state])
    return action_values

In [ ]:
def policy_iteration(environment, discount_factor=1.0, max_iterations=1e9):
    # Start with a random policy
    # num states x num actions / num actions
    n_states = environment.observation_space.n
    n_actions = environment.action_space.n
    policy = np.ones([n_states, n_actions]) / n_actions
    
    # Initialize counter of evaluated policies
    evaluated_policies = 1
    # Repeat until convergence or critical number of iterations reached
    for i in range(int(max_iterations)):
        stable_policy = True
        # Evaluate current policy
        V = policy_evaluation(policy, environment, discount_factor=discount_factor)
        # Go through each state and try to improve actions that were taken (policy Improvement)
        for state in range(n_states):
            # Choose the best action in a current state under current policy
            current_action =  policy[state].tolist().index(max(policy[state]))  
            # Look one step ahead and evaluate if current action is optimal
            # We will try every possible action in a current state
            action_value = one_step_lookahead(environment, state, V, discount_factor)
            # Select a better action
            best_action = action_value.tolist().index(max(action_value))
            # If action didn't change
            if current_action != best_action:
                stable_policy = True
                # Greedy policy update
                policy[state] = np.eye(n_actions)[best_action]
        evaluated_policies += 1
        # If the algorithm converged and policy is not changing anymore, then return final policy and value function
        if stable_policy:
            return policy, V

In [ ]:
# Number of episodes to play
n_episodes = 1000
iteration_name = "Policy iteration"
iteration_func = policy_iteration
# Load a Frozen Lake environment
environment = gym.make('FrozenLake-v1')
environment.reset()
environment.render()

# Search for an optimal policy using policy iteration
policy, V = iteration_func(environment.env)
# Apply best policy to the real environment
wins, total_reward, average_reward = run_episodes(environment, n_episodes, policy, False)
print(f'{iteration_name}: number of wins over {n_episodes} episodes = {wins}')
print(f'{iteration_name}: average reward over {n_episodes} episodes = {average_reward} \n\n')


/usr/local/lib/python3.9/dist-packages/gym/core.py:317: DeprecationWarning: WARN: Initializing wrapper in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(
/usr/local/lib/python3.9/dist-packages/gym/wrappers/step_api_compatibility.py:39: DeprecationWarning: WARN: Initializing environment in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(
/usr/local/lib/python3.9/dist-packages/gym/core.py:49: DeprecationWarning: WARN: You are calling render method, but you didn't specified the argument render_mode at environment initialization. To maintain backward compatibility, the environment will render in human mode.
If you want to render in human mode, initialize the environment in this way: gym.make('EnvName', render_mode='human') and don't 

Policy-evaluation converged at iteration #12
Policy iteration: number of wins over 1000 episodes = 438
Policy iteration: average reward over 1000 episodes = 0.438 




2.Create a Value Iteration function with the following parameters

> a.environment: Initialized OpenAI gym environment object


> b.discount_factor: MDP discount factor


> c.theta:  A  threshold  of  a  value  function  change.  Once  the  update  to  value function is below this number


> d.max_iterations: Maximum number of iterations









In [ ]:
def value_iteration(environment, discount_factor=1.0, theta=1e-9, max_iterations=1e9):

    n_states = environment.observation_space.n
    n_actions = environment.action_space.n
    # Initialize state-value function with zeros for each environment state
    V = np.zeros(n_states)
    for i in range(int(max_iterations)):
        # Early stopping condition
        delta = 0
        # Update each state
        for state in range(n_states):
            # Do a one-step lookahead to calculate state-action values
            action_value = one_step_lookahead(environment, state, V, discount_factor)
            # Select best action to perform based on the highest state-action value
            best_action_value = max(action_value)
            # Calculate change in value
            delta = max(delta, np.abs(V[state] - best_action_value))
            # Update the value function for current state
            V[state] = best_action_value
            # Check if we can stop
        if delta < theta:
            print(f'Value-iteration converged at iteration #{i}.')
            break

    # Create a deterministic policy using the optimal value function
    policy = np.zeros([n_states, n_actions])
    for state in range(n_states):
        # One step lookahead to find the best action for this state
        action_value = one_step_lookahead(environment, state, V, discount_factor)
        # Select best action based on the highest state-action value
        best_action = action_value.tolist().index(max(action_value))
        # Update the policy to perform a better action at a current state
        policy[state, best_action] = 1.0
    return policy, V

In [ ]:
# Number of episodes to play
n_episodes = 1000
iteration_name = "Value iteration"
iteration_func = value_iteration
# Load a Frozen Lake environment
environment = gym.make('FrozenLake-v1') #make environment
environment.reset()
environment.render()
# Search for an optimal policy using policy iteration
policy, V = iteration_func(environment.env)
# Apply best policy to the real environment
wins, total_reward, average_reward = run_episodes(environment, n_episodes, policy, False)
print(f'{iteration_name}: number of wins over {n_episodes} episodes = {wins}')
print(f'{iteration_name}: average reward over {n_episodes} episodes = {average_reward} \n\n')

Value-iteration converged at iteration #523.
Value iteration: number of wins over 1000 episodes = 734
Value iteration: average reward over 1000 episodes = 0.734 




3.Compare  the number of  wins, average  return  after  1000  episodes andcomment  on which method performed better.

In [ ]:
# Number of episodes to play
n_episodes = 1000
# Functions to find best policy
solvers = [('Policy Iteration', policy_iteration),
           ('Value Iteration', value_iteration)]
for iteration_name, iteration_func in solvers:
    # Load a Frozen Lake environment
    environment = gym.make('FrozenLake-v1')
    # Search for an optimal policy using policy iteration
    policy, V = iteration_func(environment.env)
    # Apply best policy to the real environment
    wins, total_reward, average_reward = run_episodes(environment, n_episodes, policy, False)
    print(f'{iteration_name} :: number of wins over {n_episodes} episodes = {wins}')
    print(f'{iteration_name} :: average reward over {n_episodes} episodes = {average_reward} \n\n')

Policy-evaluation converged at iteration #12
Policy Iteration :: number of wins over 1000 episodes = 447
Policy Iteration :: average reward over 1000 episodes = 0.447 


Value-iteration converged at iteration #523.
Value Iteration :: number of wins over 1000 episodes = 764
Value Iteration :: average reward over 1000 episodes = 0.764 




***Policy-evaluation converges faster since in the section the value of the states are compared, but in value-iteration the best-action-value is compared, which can have a bigger difference.***